#### 📌 **HİPERPARAMETRELERİ BURADAN AYARLA**

In [9]:
import os
import re
import cv2
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix, f1_score, accuracy_score, precision_score, recall_score, roc_auc_score
from tensorflow.keras.models import load_model

In [10]:
def mammo_gorunumlerini_listele(kok_dizin):#Chatgpt
    pattern = re.compile(r'^[A-Z]_\d+_\d+\.(LEFT|RIGHT)_(CC|MLO)\.jpg$', re.IGNORECASE)
    siniflar = ['benign', 'cancer', 'normal']
    yollar, etiketler = [], []
    for cls in siniflar:
        cls_yolu = os.path.join(kok_dizin, cls)
        if not os.path.isdir(cls_yolu):
            continue
        for durum_id in os.listdir(cls_yolu):
            durum_yolu = os.path.join(cls_yolu, durum_id)
            if not os.path.isdir(durum_yolu):
                continue
            for dosya_adi in os.listdir(durum_yolu):
                if pattern.match(dosya_adi):
                    yollar.append(os.path.join(durum_yolu, dosya_adi))
                    etiketler.append(cls)
    return yollar, etiketler

In [11]:
def load_and_preprocess(paths, labels, target_size, method='minmax'):
    X, y = [], []
    label_map = {'normal': 0, 'benign': 1, 'cancer': 2}
    for img_path, lbl in zip(paths, labels):
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, target_size)
        img = img.astype(np.float32)
        if method == 'minmax':
            img /= 255.0
        else:
            img = (img - img.mean()) / (img.std() + 1e-8)
        X.append(img[..., np.newaxis])
        y.append(label_map[lbl])
    return np.array(X), np.array(y)


 Metirkler**

In [12]:
if __name__ == '__main__':
    kok_dizin = os.path.join(os.getcwd(), "Project1")
    yollar, etiketler = mammo_gorunumlerini_listele(kok_dizin)
    X_test, y_test = load_and_preprocess(yollar, etiketler,target_size=(224, 224))
    model=load_model("model2.h5")
    T_m = 0.46
    T_b = 0.4
    probs = model.predict(X_test)
    classes = ['normal', 'benign', 'cancer']
    label_map = {'normal': 0, 'benign': 1, 'cancer': 2}
    preds = [2 if p[2] >= T_m else 1 if p[1] >= T_b else 0 for p in probs]
    print(classification_report(y_test, preds, target_names=classes))
    print("Confusion Matrix:\n", confusion_matrix(y_test, preds))
    print("Accuracy:", accuracy_score(y_test, preds))
    print("Precision:", precision_score(y_test, preds, average='macro'))
    print("Recall:", recall_score(y_test, preds, average='macro'))
    print("📌 F1 Score:", f1_score(y_test, preds, average='macro'))
    try:
        print("AUC:", roc_auc_score(pd.get_dummies(y_test), probs,
                                 multi_class='ovr', average='macro'))
    except:
        print("AUC hesaplanamadı.")

              precision    recall  f1-score   support

      normal       1.00      1.00      1.00        80
      benign       1.00      0.56      0.72        80
      cancer       0.70      1.00      0.82        80

    accuracy                           0.85       240
   macro avg       0.90      0.85      0.85       240
weighted avg       0.90      0.85      0.85       240

Confusion Matrix:
 [[80  0  0]
 [ 0 45 35]
 [ 0  0 80]]
Accuracy: 0.8541666666666666
Precision: 0.8985507246376812
Recall: 0.8541666666666666
📌 F1 Score: 0.8468376068376068
AUC: 0.9972135416666666
